# Homework 01 

This Jupyter Notebook file is a template that we provide for the homework.
You _must_ use this template and follow the instructions.
To unify the submission, we require you to store specific intermediate results into _"submission-variables"_.
These values are printed at the _end_ of the jupyter notebook and will be (partially) used to grade your assignment.
Please ensure that these variables have the correct values and aren't overwritten by following code-cells.

In [1]:
# used for submission variables
from copy import deepcopy

## Sentinel-2 tile download

Download a Sentinel-2 tile from [SciHub Copernicus](https://scihub.copernicus.eu/dhus/#/home) with the following requirements:
- With the sensing date _after_ 01.03.2022
- With the product-type `S2MSI2A`
- _At least_ a cloud cover percentage of 1% but _no more_ than 20%
- Not the tile with the id: 
    - `6e5ad1c9-fc0e-4b93-9b5c-d0d401673206`

_Write_ down (inside of a markdown cell) a city that is part of the downloaded Sentinel-2 tile!

To ensure that no secrets are shared, please use the following code template.
You can create a [TOML](https://en.wikipedia.org/wiki/TOML) file called `.secrets.toml` with the contents:

```toml

scihub_copernicus_user = "<YOUR_PWD>"
scihub_copernicus_pwd = "<YOUR_PWD>"
```

Or you can simply remove your username/password _after_ executing the notebook and _before_ submitting it.

In [2]:
# KEEP
from dynaconf import Dynaconf

settings = Dynaconf(
    envvar_prefix="IP4RS",
    settings_files=["settings.toml", ".secrets.toml"],
)

SENTINEL_API_URL = "https://scihub.copernicus.eu/dhus"
YOUR_USERNAME = "kton"
YOUR_PWD = "12345678"
user = settings.get("scihub_copernicus_user", default=YOUR_USERNAME)
pwd = settings.get("scihub_copernicus_pwd", default=YOUR_PWD)

assert user != r""
assert pwd != r""
###

In [4]:
# imports
import numpy as np
import geopandas
from pathlib import Path
import osmnx


from sentinelsat import SentinelAPI
from datetime import date
from shapely.geometry import Point
import rasterio
import zipfile

from enum import Enum
from typing import List, Sequence
import matplotlib.pyplot as plt

import rasterio.mask


In [5]:
# Get lats and longs
latitude = 52.112230
longitude = 13.327135



In [6]:
series = geopandas.GeoSeries([Point(longitude, latitude)], crs="EPSG:4326")
series



0    POINT (13.32714 52.11223)
dtype: geometry

In [7]:
api_sent = SentinelAPI(user, pwd, SENTINEL_API_URL)
# api_sent.query?

In [8]:
series.to_wkt()


0    POINT (13.327135 52.11223)
dtype: object

In [9]:
poi = series.to_wkt()[0]
poi



'POINT (13.327135 52.11223)'

In [10]:
# Set params
start_date = date(year=2022, month=3, day=1)
end_date = date(year=2022, month=4, day=30)
satellite = "Sentinel-2"
producttype = "S2MSI2A"

products = api_sent.query(
    poi,
    date=(start_date, end_date),
    platformname=satellite,
    producttype=producttype,
    cloudcoverpercentage=(1, 20),
)

# convert to GeoPandas GeoDataFrame
products_gdf = api_sent.to_geodataframe(products)
assert not products_gdf.empty
products_gdf.head(1)
# If you get an internal-service error, simply try again




,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid,geometry
0de0c49b-72d0-4ff9-9605-79c51399d676,S2B_MSIL2A_20220423T100549_N0400_R022_T33UUT_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-04-23T10:05:49.024Z, Instrument: MS...",false,2022-04-23 13:43:29,2022-04-23 10:05:49.024,2022-04-23 10:05:49.024,2022-04-23 17:44:37.025,...,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS2_20220423T134329_S2022...,S2B_OPER_MSI_L2A_TL_VGS2_20220423T134329_A0267...,S2B_MSIL2A_20220423T100549_N0400_R022_T33UUT_2...,0de0c49b-72d0-4ff9-9605-79c51399d676,"MULTIPOLYGON (((12.12921 51.32805, 13.70458 51..."


In [11]:
# The tile-id should be generated here
# for example:
tile_id = "0de0c49b-72d0-4ff9-9605-79c51399d676"



In [12]:
# sort and select the first row
product = products_gdf.sort_values(
    ["cloudcoverpercentage", "ingestiondate"], ascending=[True, True]
).head(1)
product
print(product["title"][0])

S2B_MSIL2A_20220324T100649_N0400_R022_T32UQC_20220324T132305


In [13]:
# ensuring that the tile is the one we are also providing via a direct
# download link
assert product["title"][0] == "S2B_MSIL2A_20220324T100649_N0400_R022_T32UQC_20220324T132305"

In [14]:
# Would fail, because there are entries that cannot be JSON encoded!
# product.explore()
# Only visualize the relevant columns and convert them if necessary
product[["title", "summary", "geometry"]].explore()

In [15]:
product

,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid,geometry
bbd5d330-b017-4ae3-b669-2a2a0768272b,S2B_MSIL2A_20220324T100649_N0400_R022_T32UQC_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-03-24T10:06:49.024Z, Instrument: MS...",false,2022-03-24 13:23:05,2022-03-24 10:06:49.024,2022-03-24 10:06:49.024,2022-03-24 16:36:45.886,...,S2MSI2A,2017-013A,DESCENDING,Sentinel-2B,Level-2A,S2B_OPER_MSI_L2A_DS_VGS2_20220324T132305_S2022...,S2B_OPER_MSI_L2A_TL_VGS2_20220324T132305_A0263...,S2B_MSIL2A_20220324T100649_N0400_R022_T32UQC_2...,bbd5d330-b017-4ae3-b669-2a2a0768272b,"MULTIPOLYGON (((13.44210 51.27893, 13.53998 52..."


In [17]:
#Download Tile

download_dir = Path("./data/homework")
download_dir.mkdir(exist_ok=True)

# ~800MB large!
# May take a while to download
# The download is very slow from their server

output_file = download_dir / f"{product.title[0]}.zip"
if not output_file.exists():
    api_sent.download_all(product.index, directory_path=download_dir)

MD5 checksumming:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

In [22]:
# extract zip
zipf = zipfile.ZipFile(output_file)
zipf.extractall(path="data/homework")

In [23]:
unzipped_dir = output_file.with_suffix(".SAFE")
assert unzipped_dir.exists()

In [39]:
### DO NOT EDIT
submission_tile_id = deepcopy(tile_id)
##

## Valid sub-region

From the previously downloaded tile, extract a _valid_ subregion.
Convert the subregion to a `uint8` array without dropping important information (i.e., not simply calling `np.uint8` on the input).

In [ ]:
### Add here the code to select a valid subregion

subregion_uint8 = ...

In [ ]:
### DO NOT EDIT
submission_subregion_uint8 = deepcopy(subregion_uint8)
###

### Visualize the valid sub-region

Visualize a _single band_ from the valid subregion _without_ applying any explicit or implicit normalization strategy.
The input array passed to `plt.imshow` has to be in the range `[0, 255]`. This range should also be reflected in the `vmin/vmax` parameters.

In [ ]:
# Add the code to visualize the single band
# without normalization here:

# imshow_inp is the array that should be passed to plt.imshow
imshow_inp = ...

In [ ]:
# DO NOT EDIT
submission_imshow_inp = deepcopy(imshow_inp)
###

## Morphological operators & Thresholding

The goal of this section is to combine morphological operators with thresholding strategies to create a binary/output mask that can then be used to segment the region/object of interest.
You are _not_ required to perfectly segment the object of interest. It is sufficient if the visual results _indicate_ the object of interest.

You have to meet the following requirements:
- You are _not_ allowed to use simple rectangular structuring elements.
- The values of the output mask must either be 0 or 1
    - 0 is used to indicate the absence of the object
    - 1 is used to indicate the presence of the object

In [ ]:
# Add code here to apply morphological operators
# and thresholding strategy
# You are allowed to change/mix the order

output_mask = ...

In [ ]:
# DO NOT EDIT
submission_output_mask = deepcopy(output_mask)
###

To visually evaluate your result, use the output mask as an overlay for the RGB channels of the input array.
The colored output should be normalized to enhance the viewing experiment.
You can choose any _valid_ normalization strategy/method for the RGB channels.

In [ ]:
# Visualize the overlayed image

### Discussion

Discuss what you have learned when applying the morphological operators and thresholding strategies.
What didn't work? What issues did you have? What helped?
How did you decide to use the variables you have chosen?

It is not necessary to give an in-depth discussion about each value.
It should be clear, however, that you have tried different configurations and why you have decided on the current configuration.

Here you should write discussion:

## Submission

- Follow the instructions mentioned in this notebook.
- You are _not_ allowed to install other libraries.
    - The submitted jupyter notebook should be able to run as-is on paperspace and/or the provided Docker image.
- Explain the steps that went into the decision making of the specific values.
- It is essential that each homework should be done individually!
- The notebook should have been executed once before submitting it
    - The notebook should _not_ be _cleaned_ before submitting

Each student should submit the following materials to the ISIS system:
- One `zip` file containing the jupyter notebook named as: `IP4RS2022-HW1_<NAME_SURNAME>.zip`



### Submission variables

Please ensure that these variables are correct after you have restarted your notebook and executed all cells from top to bottom.
You are _not allowed_ to modify the following cells:

In [ ]:
import rich

rich.print(f"{submission_tile_id=}")
rich.print(f"{submission_subregion_uint8=}")
rich.print(f"{submission_output_mask=}")